<a href="https://colab.research.google.com/github/Nikhileswar-Komati/100D_ML/blob/master/Hackathon/JanataHack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import files
files.upload()

In [ ]:
! unzip '/content/Train_hMYJ020.zip'
! unzip '/content/Test_ND2Q3bm.zip'

Archive:  /content/Train_hMYJ020.zip
  inflating: train_data_dict.csv     
   creating: __MACOSX/
  inflating: __MACOSX/._train_data_dict.csv  
  inflating: train.csv               
  inflating: __MACOSX/._train.csv    
Archive:  /content/Test_ND2Q3bm.zip
  inflating: test.csv                
  inflating: __MACOSX/._test.csv     


In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [ ]:
dicts = pd.read_csv('/content/train_data_dict.csv')
dicts

,Column,Description
0,case_id,Case_ID registered in Hospital
1,Hospital_code,Unique code for the Hospital
2,Hospital_type_code,Unique code for the type of Hospital
3,City_Code_Hospital,City Code of the Hospital
4,Hospital_region_code,Region Code of the Hospital
5,Available Extra Rooms in Hospital,Number of Extra rooms available in the Hospital
6,Department,Department overlooking the case
7,Ward_Type,Code for the Ward type
8,Ward_Facility_Code,Code for the Ward Facility
9,Bed Grade,Condition of Bed in the Ward


In [ ]:
train['source'] = 'train'
test['source'] = 'test'
data = pd.concat([train, test])
print(data.shape, train.shape, test.shape)
data.sample(5)

(455495, 19) (318438, 19) (137057, 18)


,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay,source
288537,288538,11,b,2,Y,3,gynecology,Q,D,3.0,70002,15.0,Urgent,Moderate,8,21-30,4717.0,More than 100 Days,train
355,356,12,a,9,Y,4,gynecology,Q,B,2.0,79116,9.0,Urgent,Moderate,3,31-40,4988.0,21-30,train
77663,396102,6,a,6,X,3,radiotherapy,R,F,2.0,44665,7.0,Emergency,Moderate,4,0-10,3250.0,NaN,test
38358,38359,27,a,7,Y,3,radiotherapy,S,C,3.0,59482,8.0,Emergency,Moderate,4,11-20,4752.0,31-40,train
28949,347388,12,a,9,Y,6,gynecology,Q,B,2.0,119871,12.0,Urgent,Moderate,4,71-80,6496.0,NaN,test


In [ ]:
data.isnull().sum()

case_id                                   0
Hospital_code                             0
Hospital_type_code                        0
City_Code_Hospital                        0
Hospital_region_code                      0
Available Extra Rooms in Hospital         0
Department                                0
Ward_Type                                 0
Ward_Facility_Code                        0
Bed Grade                               148
patientid                                 0
City_Code_Patient                      6689
Type of Admission                         0
Severity of Illness                       0
Visitors with Patient                     0
Age                                       0
Admission_Deposit                         0
Stay                                 137057
source                                    0
dtype: int64

In [ ]:
from scipy.stats import mode
impute_bed_grades = data.pivot_table('Bed Grade', index = ['Available Extra Rooms in Hospital', 'Ward_Type', 'Ward_Facility_Code'], aggfunc = (lambda x: mode(x).mode[0]))
count = 0
for i, row in data.loc[data['Bed Grade'].isnull(), :].iterrows():
  x = (row['Available Extra Rooms in Hospital'], row['Ward_Type'], row['Ward_Facility_Code'])
  if x in impute_bed_grades.index:
    count += 1
    data.loc[i,'Bed Grade'] = impute_bed_grades.loc[x].values[0]
print("IMPUTED VALUES", count)

IMPUTED VALUES 148


In [ ]:
data.loc[data['City_Code_Patient'].isnull(), ['patientid', 'City_Code_Patient']]['patientid'].value_counts()

16638    19
90620    17
8265     16
74285    15
56688    12
         ..
59505     1
18425     1
10237     1
65534     1
34817     1
Name: patientid, Length: 2436, dtype: int64

In [ ]:
from scipy.stats import mode
impute_patient_city_codes = data.pivot_table('City_Code_Patient', index = ['City_Code_Hospital', 'Hospital_code'], aggfunc = (lambda x: mode(x).mode[0]))
count = 0
for i, row in data.loc[data['City_Code_Patient'].isnull(), :].iterrows():
  x = (row['City_Code_Hospital'], row['Hospital_code'])
  if x in impute_patient_city_codes.index:
    count += 1
    data.loc[i,'City_Code_Patient'] = impute_patient_city_codes.loc[x].values[0]
print("IMPUTED VALUES", count)

IMPUTED VALUES 6689


In [ ]:
data.isnull().sum()

case_id                                   0
Hospital_code                             0
Hospital_type_code                        0
City_Code_Hospital                        0
Hospital_region_code                      0
Available Extra Rooms in Hospital         0
Department                                0
Ward_Type                                 0
Ward_Facility_Code                        0
Bed Grade                                 0
patientid                                 0
City_Code_Patient                         0
Type of Admission                         0
Severity of Illness                       0
Visitors with Patient                     0
Age                                       0
Admission_Deposit                         0
Stay                                 137057
source                                    0
dtype: int64

In [ ]:
data.describe()

,case_id,Hospital_code,City_Code_Hospital,Available Extra Rooms in Hospital,Bed Grade,patientid,City_Code_Patient,Visitors with Patient,Admission_Deposit
count,455495.00000,455495.000000,455495.000000,455495.000000,455495.000000,455495.000000,455495.000000,455495.000000,455495.000000
mean,227748.00000,18.326335,4.767798,3.196140,2.628207,65786.793563,7.246455,3.284229,4877.434022
std,131490.22477,8.634036,3.102450,1.166994,0.871880,37968.830850,4.714925,1.768044,1084.982089
min,1.00000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1800.000000
25%,113874.50000,11.000000,2.000000,2.000000,2.000000,32874.000000,4.000000,2.000000,4184.000000
50%,227748.00000,19.000000,5.000000,3.000000,3.000000,65735.000000,8.000000,3.000000,4738.000000
75%,341621.50000,26.000000,7.000000,4.000000,3.000000,98576.500000,8.000000,4.000000,5405.000000
max,455495.00000,32.000000,13.000000,24.000000,4.000000,131624.000000,38.000000,32.000000,11920.000000


In [ ]:
data.loc[data['patientid'] == 16638, ['patientid', 'City_Code_Patient', 'source']]

,patientid,City_Code_Patient,source
73053,16638,8.0,test
73054,16638,8.0,test
73055,16638,2.0,test
73056,16638,8.0,test
73057,16638,2.0,test
73058,16638,8.0,test
73059,16638,8.0,test
73060,16638,8.0,test
73061,16638,8.0,test
73062,16638,8.0,test


In [ ]:
table = data.pivot_table('City_Code_Patient', index = ['patientid'], aggfunc = (lambda x: mode(x).mode[0]))
data['City_Code_Patient_mode'] = data['patientid'].apply(lambda x: table.loc[x])

In [ ]:
data.loc[data['patientid'] == 16638, ['patientid', 'City_Code_Patient', 'City_Code_Patient_mode']]

,patientid,City_Code_Patient,City_Code_Patient_mode
73053,16638,8.0,8.0
73054,16638,8.0,8.0
73055,16638,2.0,8.0
73056,16638,8.0,8.0
73057,16638,2.0,8.0
73058,16638,8.0,8.0
73059,16638,8.0,8.0
73060,16638,8.0,8.0
73061,16638,8.0,8.0
73062,16638,8.0,8.0


In [ ]:
data.sample(5)

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay,source,City_Code_Patient_mode
75072,75073,25,e,1,X,2,anesthesia,R,E,3.0,105887,8.0,Emergency,Moderate,2,61-70,6041.0,11-20,train,8.0
154874,154875,1,d,10,Y,2,gynecology,S,B,1.0,112033,8.0,Trauma,Extreme,4,61-70,4284.0,51-60,train,8.0
27998,346437,10,e,1,X,2,gynecology,S,E,2.0,97522,10.0,Trauma,Moderate,4,61-70,4413.0,NaN,test,10.0
316365,316366,25,e,1,X,2,gynecology,S,E,3.0,44128,7.0,Urgent,Extreme,2,11-20,3103.0,31-40,train,7.0
222357,222358,26,b,2,Y,4,radiotherapy,Q,D,4.0,23878,7.0,Urgent,Moderate,4,31-40,5660.0,21-30,train,7.0


In [ ]:
data.dtypes

case_id                                int64
Hospital_code                          int64
Hospital_type_code                    object
City_Code_Hospital                     int64
Hospital_region_code                  object
Available Extra Rooms in Hospital      int64
Department                            object
Ward_Type                             object
Ward_Facility_Code                    object
Bed Grade                            float64
patientid                              int64
City_Code_Patient                    float64
Type of Admission                     object
Severity of Illness                   object
Visitors with Patient                  int64
Age                                   object
Admission_Deposit                    float64
Stay                                  object
source                                object
City_Code_Patient_mode               float64
dtype: object

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['Encoded_Age'] = le.fit_transform(data['Age'])
data = pd.get_dummies(data, columns=['Encoded_Age', 'Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type', 'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness'])

In [ ]:
data['Stay'].unique()

array(['0-10', '41-50', '31-40', '11-20', '51-60', '21-30', '71-80',
       'More than 100 Days', '81-90', '61-70', '91-100', nan],
      dtype=object)

In [ ]:
data.dtypes

case_id                                int64
Hospital_code                          int64
City_Code_Hospital                     int64
Available Extra Rooms in Hospital      int64
Bed Grade                            float64
patientid                              int64
City_Code_Patient                    float64
Visitors with Patient                  int64
Age                                   object
Admission_Deposit                    float64
Stay                                  object
source                                object
City_Code_Patient_mode               float64
Encoded_Age_0                          uint8
Encoded_Age_1                          uint8
Encoded_Age_2                          uint8
Encoded_Age_3                          uint8
Encoded_Age_4                          uint8
Encoded_Age_5                          uint8
Encoded_Age_6                          uint8
Encoded_Age_7                          uint8
Encoded_Age_8                          uint8
Encoded_Ag

In [ ]:
len(data.columns)

56

In [ ]:
data.drop(['Age', 'City_Code_Patient', 'patientid'],axis=1,inplace=True)
data['Admission_Deposit'] = np.log(data['Admission_Deposit'])
data['Available Extra Rooms in Hospital'] = np.log(data['Available Extra Rooms in Hospital'] + 1)

train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]
#Drop unnecessary columns:
test.drop(['Stay','source'],axis=1,inplace=True)
train.drop(['source'],axis=1,inplace=True)

train.to_csv("train_modified.csv",index=False)
test.to_csv("test_modified.csv",index=False)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
train['Stay'].unique()

array(['0-10', '41-50', '31-40', '11-20', '51-60', '21-30', '71-80',
       'More than 100 Days', '81-90', '61-70', '91-100'], dtype=object)

In [ ]:
from google.colab import files
files.upload()

Saving train_modified.csv to train_modified.csv
Saving test_modified.csv to test_modified.csv


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
train = pd.read_csv('/content/train_modified.csv')
test = pd.read_csv('/content/test_modified.csv')


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['Stay'] = le.fit_transform(train['Stay'])

In [ ]:
train['Stay'].unique()

In [ ]:
#Define target and ID columns:
target = 'Stay'
IDcol = ['case_id']
predictors = [x for x in train.columns if x not in [target] + IDcol]
from sklearn import model_selection, metrics
def modelfit(alg, dtrain, dtest, predictors, target, IDcol, filename):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target])
    print("Fit Done")   
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    print("Predict Done")
    print(dtrain_predictions)

    #Perform cross-validation:
    cv_score = model_selection.cross_val_score(alg, dtrain[predictors], dtrain[target], scoring='accuracy')
    
    #Print model report:
    print("\nModel Report")
    print("ACC : %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions))
    print("CV Score : Mean - %.4g | Std - %.4g | Min - %.4g | Max - %.4g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
    
    #Predict on testing data:
    dtest[target] = le.inverse_transform(alg.predict(dtest[predictors]))
    
    #Export submission file:
    IDcol.append(target)
    submission = pd.DataFrame({ x: dtest[x] for x in IDcol})
    submission.to_csv(filename, index=False)

In [ ]:
from lightgbm import LGBMClassifier

predictors = [x for x in train.columns if x not in [target] + IDcol]
alg4 = LGBMClassifier(learning_rate = 0.06, max_depth = 50, n_estimators = 395, objective = 'multiclass', boosting_type = 'gbdt')
modelfit(alg4, train, test, predictors, target, IDcol, 'alg4.csv')

Fit Done
Predict Done
[2 2 2 ... 1 3 1]

Model Report
ACC : 0.4463


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
